# Part 2: Deploying and Querying a Model


### Step 1: Import Torch, PySyft, and Grid

In [1]:
import torch as th
from torch import nn
import torch.nn.functional as F
import syft as sy
import grid as gr

hook = sy.TorchHook(th)
hook.local_worker.is_client_worker = False
me = hook.local_worker

### Step 2: Create and trace a model


We'll implement a simplified version of the model implemented in [this tutorial](https://towardsdatascience.com/skin-cancer-classification-with-machine-learning-c9d3445b2163) by Nyla Pirani.

In [2]:
# import torchvision.models as models
# model = models.resnet50(pretrained=True)

In [3]:
# Instatiate model
# model = Net()

# Trace the model
# traced_model = th.jit.trace(model, th.tensor([1.0, 2]))

#### Step 3:  Training the model

In [4]:
# A Toy Model
class Net(th.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = th.nn.Linear(2, 3)
        self.fc2 = th.nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instatiate model
model = Net()

# Trace the model
traced_model = th.jit.trace(model, th.tensor([1.0, 2]))

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)


def train():
    # Training Logic
    opt = th.optim.SGD(params=model.parameters(),lr=0.1)
    for epoch in range(20):
        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print('Epoch %d:' % (epoch + 1), loss.data)
        
train()

Epoch 1: tensor(7.3171)
Epoch 2: tensor(1.1707)
Epoch 3: tensor(0.9776)
Epoch 4: tensor(0.9454)
Epoch 5: tensor(0.8751)
Epoch 6: tensor(0.7873)
Epoch 7: tensor(0.6785)
Epoch 8: tensor(0.5821)
Epoch 9: tensor(0.4791)
Epoch 10: tensor(0.3886)
Epoch 11: tensor(0.2854)
Epoch 12: tensor(0.2066)
Epoch 13: tensor(0.1629)
Epoch 14: tensor(0.1120)
Epoch 15: tensor(0.0864)
Epoch 16: tensor(0.0667)
Epoch 17: tensor(0.0450)
Epoch 18: tensor(0.0433)
Epoch 19: tensor(0.0241)
Epoch 20: tensor(0.0275)


#### Step 4: Deploy websocket worker

In [5]:
worker = gr.GridClient("http://127.0.0.1:5000")#"https://websocketgridtest2.herokuapp.com/",id="grid")

#### Step 5: Host model

In [6]:
traced_model.send(worker)

not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)
not enough values to unpack (expected 10, got 3)


TypeError: 'NoneType' object is not subscriptable

#### Step 6: Query model

In [ ]:
x = th.tensor([1,2,3.,4]).send(worker)
x

In [ ]:
y = x + x
y

In [ ]:
y.get()

In [ ]:
model = th.nn.Linear(4, 1)
model.send(worker)

In [ ]:
model(x).get()

### Step 8: Reconnecting to Same Node

If you run the same connect command that you did before, it will just connect to the existing node instead of launching it.

In [ ]:
worker = gr.GridClient(addr=gr.launch_on_heroku("opengrid10", verbose=True, check_deps=True))

In [ ]:
x = th.tensor([1,2,3,4]).send(worker)
x

#### Useful Tip

You can set check_deps=False and it'll run a little faster

In [ ]:
worker = gr.GridClient(addr=gr.launch_on_heroku("opengrid10", verbose=True, check_deps=False))

#### Useful Tip

Or you can just enter the address yourself

In [ ]:
worker = gr.GridClient(addr="https://opengrid10.herokuapp.com")

x = th.tensor([1,2,3,4]).send(worker)
x

### Step 9: Destroying a Node

Destroying the heroku node, deleting all objects (all tensors) is as simple as calling .destroy().

In [ ]:
worker.destroy()